In [1]:
using RigidBodySim
using RigidBodyDynamics
using ValkyrieRobot
using OrdinaryDiffEq
using DiffEqCallbacks
using RigidBodyTreeInspector
using ValkyrieRobot.BipedControlUtil
using RigidBodyDynamics.Contact
using RigidBodyDynamics.PDControl
using MomentumBasedControl
using StaticArrays
using RigidBodyTreeInspector, DrakeVisualizer
using ValkyrieRobot, ValkyrieRobot.BipedControlUtil
using ForwardDiff

INFO: Recompiling stale cache file /home/twan/code/RigidBodyDynamics/lib/v0.6/MomentumBasedControl.ji for module MomentumBasedControl.


In [2]:
val = Valkyrie()
mechanism = val.mechanism;
remove_fixed_tree_joints!(mechanism);

In [3]:
# add environment
rootframe = root_frame(mechanism)
floor = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism, floor);

In [4]:
# Create visualizer
any_open_visualizer_windows() || (new_visualizer_window(); sleep(1));
vis = Visualizer()[:valkyrie]
setgeometry!(vis, mechanism, RigidBodyTreeInspector.parse_urdf(ValkyrieRobot.urdfpath(), mechanism; package_path = [ValkyrieRobot.packagepath()]))

In [5]:
feet = Dict(:left => findbody(mechanism, "leftFoot"), :right => findbody(mechanism, "rightFoot"))
hands = Dict(:left => findbody(mechanism, "leftPalm"), :right => findbody(mechanism, "rightPalm"));

In [6]:
state = MechanismState(mechanism)
controllerstate = MomentumBasedControllerState(state);

In [7]:
function initialize(state::MechanismState, controllerstate::MomentumBasedControllerState, val::Valkyrie, vis::Visualizer)
    zero!(state)
    for side in instances(Side)
        set_configuration!(state, findjoint(mechanism, "$(side)KneePitch"), [1.205])
        set_configuration!(state, findjoint(mechanism, "$(side)HipPitch"), [-0.49])
        set_configuration!(state, findjoint(mechanism, "$(side)AnklePitch"), [-0.71])
        set_configuration!(state, findjoint(mechanism, "$(side)ShoulderPitch"), [0.300196631343025])
        set_configuration!(state, findjoint(mechanism, "$(side)ShoulderRoll"), [flipsign_if_right(-1.25, side)])
        set_configuration!(state, findjoint(mechanism, "$(side)ElbowPitch"), [flipsign_if_right(-0.785398163397448, side)])
        set_configuration!(state, findjoint(mechanism, "$(side)ForearmYaw"), [1.571])
    end
#     kneebend = 1.1
#     hipbendextra = 0.1
#     for side in instances(Side)
#         set_configuration!(state, val.knees[side], [kneebend])
#         set_configuration!(state, val.hippitches[side], [-kneebend / 2 + hipbendextra])
#         set_configuration!(state, val.anklepitches[side], [-kneebend / 2 - hipbendextra])
#     end
    set_configuration!(state, val.basejoint, [1; 0; 0; 0; 0; 0; 1.025])
    settransform!(vis, state)
    MomentumBasedControl.reset!(controllerstate)
    nothing
end

initialize (generic function with 1 method)

In [8]:
# τ = Vector{Float64}(num_velocities(mechanism))
# result = DynamicsResult{Float64}(mechanism)
# function damped_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
#     τ[:] = velocity(state)
#     scale!(τ, -1.0)
#     dynamics!(result, state, τ)
#     copy!(vd, result.v̇)
#     copy!(sd, result.ṡ)
# end

In [9]:
initialize(state, controllerstate, val, vis)

In [10]:
controller = MomentumBasedController(mechanism, 1 / 300); # FIXME: should user PeriodicController
contacts = add_mechanism_contacts!(controller)
jointacceltasks = add_mechanism_joint_accel_tasks!(controller);

In [11]:
comgains = PDGains(10., 3.)

RigidBodyDynamics.PDControl.PDGains{Float64,Float64}(10.0, 3.0)

In [12]:
# Foot accelerations w.r.t. world
footacceltasks = Dict{Side, SpatialAccelerationTask{Float64}}()
nv = num_velocities(mechanism)
legjoints = []
for side in instances(Side)
    foot = val.feet[side]
    world_to_foot = path(mechanism, root_body(mechanism), foot)
    pelvis_to_foot = path(mechanism, val.pelvis, foot)
    append!(legjoints, pelvis_to_foot)
    frame = default_frame(foot)
    task = footacceltasks[side] = SpatialAccelerationTask(nv, world_to_foot, frame, eye(3, 3), eye(3, 3))
    add!(controller, task)
end

In [13]:
# Pelvis acceleration w.r.t. world
pelvisacceltask = SpatialAccelerationTask(nv, path(mechanism, root_body(mechanism), val.pelvis), default_frame(val.pelvis), eye(3, 3), zeros(0, 3))
add!(controller, pelvisacceltask)
pelvisgains = PDGains(10., 2.)
;

In [14]:
#Position control tasks
revolutejoints = filter(j -> joint_type(j) isa Revolute, joints(mechanism))
positioncontroltasks = Dict(joint => jointacceltasks[joint] for joint in setdiff(revolutejoints, legjoints))
positioncontrolgains = Dict(joint => PDGains(100.0, 20.) for joint in keys(positioncontroltasks))
;

In [15]:
# Position control references
references = Dict{Joint{Float64}, Any}(joint => zero for joint in keys(positioncontroltasks))
references[findjoint(mechanism, "rightShoulderRoll")] = t -> π/2 - 0.1
references[findjoint(mechanism, "rightShoulderYaw")] = t -> 1.
references[findjoint(mechanism, "leftElbowPitch")] = t -> -0.4
references[findjoint(mechanism, "leftWristPitch")] = t -> 0.5
references[findjoint(mechanism, "leftShoulderRoll")] = t -> 0.9 + 0.3 * sin(t)

(::#23) (generic function with 1 method)

In [16]:
for (joint, ref) in references
    configuration(state, joint)[:] = ref(0.)
    velocity(state, joint)[:] = ForwardDiff.derivative(ref, 0.)
end
setdirty!(state)
settransform!(vis, state);

In [17]:
feet = val.feet
com0 = center_of_mass(state)
com0 += FreeVector3D(com0.frame, 0.0, 0.0, -0.05)
μ = 0.8
normal = floor.outward_normal

FreeVector3D in "world": [0.0, 0.0, 1.0]

In [18]:
# FIXME: referencing global variables
function high_level_control(τ::AbstractVector, t::Number, state::MechanismState)
        # TODO: extract out: highlevelcontrol
    # Foot accelerations
    T = eltype(controller)
    MomentumBasedControl.reset!(controller)
    regularize_joint_accels!(controller, 0.05)
    for (side, task) in footacceltasks
        zero!(task, Inf)
        
        body = RigidBodyDynamics.Graphs.target(task.path)
        for contactsettings in contacts[body]
            set!(contactsettings, 1e-1, μ, transform(state, normal, default_frame(body)))
        end
    end
    
    # Pelvis orientation control
    Hpelvis = transform_to_root(state, val.pelvis)
    Tpelvis = transform(twist_wrt_world(state, val.pelvis), inv(Hpelvis))
    ωdpelvis = pd(pelvisgains, rotation(Hpelvis), Tpelvis.angular)
    Ṫpelvis = SpatialAcceleration(Tpelvis.body, Tpelvis.base, Tpelvis.frame, ωdpelvis, zero(ωdpelvis))
    set!(pelvisacceltask, Ṫpelvis, Inf)
    
    # Joint position control
    for (joint, task) in positioncontroltasks
        gains = positioncontrolgains[joint]
        ref = references[joint]
        qjoint, vjoint, vdjoint = val_deriv_deriv2(ref, t)
        accel = vdjoint = pd(gains, configuration(state, joint), qjoint, velocity(state, joint), vjoint)
        set!(task, accel, Inf)
    end
    
    # Centroidal momentum control
    pd_center_of_mass!(controller, comgains, state, com0, FreeVector3D(T, com0.frame), 1.0)

    τ[:] = control(controller, t, controllerstate)
    τ
end

high_level_control (generic function with 1 method)

In [19]:
initialize(state, controllerstate, val, vis)
final_time = 1.
problem = ODEProblem(state, (0., final_time), high_level_control)
@time sol = solve(problem, Tsit5(), abs_tol = 1e-8, dt = controller.Δt, callback = CallbackSet(vis, state, max_fps = 30.));

 38.510373 seconds (40.36 M allocations: 24.420 GiB, 10.69% gc time)


In [20]:
animate(vis, state, sol)